In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


# **Load Data**

In [2]:
df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
df.head(3)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False


In [3]:
print(f"DataFrame of SpaceShip value is ({df.shape})")

DataFrame of SpaceShip value is ((8693, 14))


***Data separation***

In [4]:
Y = df['Transported']       # Use of Y for fit Y train in model
Y = Y.map({False:0,True:1}) # change bool value to num 0,1
df = df.drop(labels=['PassengerId','Name','Transported'],axis='columns')

In [5]:
types = list(set(df.dtypes.tolist()))
print(f"this is all format in list of data => {types}")

this is all format in list of data => [dtype('float64'), dtype('O')]


In [6]:
#this part I break dataframe 2 section , numerical & object

df_num = df.select_dtypes(include='float64')
df_num = df_num.fillna(0)

df_num.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,39.0,0.0,0.0,0.0,0.0,0.0
1,24.0,109.0,9.0,25.0,549.0,44.0
2,58.0,43.0,3576.0,0.0,6715.0,49.0
3,33.0,0.0,1283.0,371.0,3329.0,193.0
4,16.0,303.0,70.0,151.0,565.0,2.0


In [7]:
df_obj = df.select_dtypes(include='O')

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_obj = df_obj.apply(le.fit_transform)

df_obj.head()


,HomePlanet,CryoSleep,Cabin,Destination,VIP
0,1,0,149,2,0
1,0,0,2184,2,0
2,1,0,1,2,1
3,1,0,1,2,0
4,0,0,2186,2,0


In [8]:
df = pd.concat([df_num,df_obj],axis='columns')

df.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet,CryoSleep,Cabin,Destination,VIP
0,39.0,0.0,0.0,0.0,0.0,0.0,1,0,149,2,0
1,24.0,109.0,9.0,25.0,549.0,44.0,0,0,2184,2,0
2,58.0,43.0,3576.0,0.0,6715.0,49.0,1,0,1,2,1
3,33.0,0.0,1283.0,371.0,3329.0,193.0,1,0,1,2,0
4,16.0,303.0,70.0,151.0,565.0,2.0,0,0,2186,2,0


# **Preprocessing**

***Normalize***

In [9]:
from sklearn.preprocessing import Normalizer

scaler = Normalizer()
X = scaler.fit_transform(df)

***Splite data to Train / Test***

In [10]:
from sklearn.model_selection import train_test_split

x_train , x_test , y_train , y_test = train_test_split(X,Y , test_size=0.3)

# **Create Model**

***DecisionTree Model***

In [11]:
from sklearn.tree import DecisionTreeClassifier

model_tree = DecisionTreeClassifier(criterion='gini',max_depth=10,min_samples_leaf=2)
model_tree.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=10, min_samples_leaf=2)

In [12]:
pred_tree = model_tree.predict(x_test)
from sklearn.metrics import accuracy_score

acc_tree = accuracy_score(y_true=y_test , y_pred=pred_tree)
print(f"Accuracy score for Decision Model is --> {acc_tree} ")

Accuracy score for Decision Model is --> 0.7707055214723927 


In [13]:
from sklearn import svm

model_svm = svm.SVC(kernel='poly',degree=6)
model_svm.fit(x_train,y_train)

SVC(degree=6, kernel='poly')

In [14]:
pred_svm = model_svm.predict(x_test)
acc_svm = accuracy_score(y_true=y_test , y_pred=pred_svm)
print(f"Accuracy score for SVM Model is --> {acc_svm} ")

Accuracy score for SVM Model is --> 0.7883435582822086 


**accuracy score with other model :**

* DecisionForest = 0.7615030674846626
* SVM = 0.785659509202454
* K_Neighbors = 0.7841257668711656
* GaussianNB = 0.5881901840490797

# **Load Main data for predict**

In [15]:
df_main = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
df_main.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [16]:
PassengerId = df_main['PassengerId'] #Separation of PassengerId from other data 

df_main = df_main.drop(labels=['PassengerId','Name'],axis='columns')

df_num_ = df_main.select_dtypes(include='float64')
df_num_ = df_num.fillna(0)
df_obj_ = df_main.select_dtypes(include='O')
df_obj_ = df_obj.apply(le.fit_transform)
df = pd.concat([df_num_,df_obj_],axis='columns')
scaler = Normalizer()

main_data = scaler.fit_transform(df)

**Pridict Main Data**

In [17]:
pred_dtree = model_tree.predict(main_data)

pred_dtree = pd.DataFrame({'Transported':pred_dtree})

pred_dtree = pred_dtree['Transported'].map({1:True,0:False}) #convert 0,1 to Boolean value

pred_dtree = pd.DataFrame({'PassengerId':PassengerId,'Transported':pred_dtree})

pred_dtree.head()         

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


In [18]:
pred_svm = model_svm.predict(main_data)

pred_svm = pd.DataFrame({'Transported':pred_svm})

pred_svm = pred_svm['Transported'].map({1:True,0:False}) #convert 0,1 to Boolean value

pred_svm = pd.DataFrame({'PassengerId':PassengerId,'Transported':pred_svm})

pred_svm.head()    

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


In [19]:
pred_svm.to_csv('submission_SVM.csv',index=False)